In [10]:
import pickle

def read_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

# 使用示例
file_path = 'H:/NERF/data/USPTO-15K/valid_1_data.pickle'
loaded_data = read_pickle_file(file_path)

# 处理读取的数据
print(loaded_data)


[{'element': array([ 6,  8,  6,  6,  6,  7,  6,  7,  6,  8,  6,  7,  6,  6,  6,  8,  6,
        6,  6,  8,  6,  8, 17,  6,  6,  6,  7,  6,  6]), 'src_bond': array([[ 1,  0,  0,  0,  0,  0],
       [ 0,  2,  1,  1,  1,  1],
       [ 1,  3, 14,  2,  2,  2],
       [ 2,  4,  3,  3,  3,  3],
       [ 3,  5, 12,  4,  4,  4],
       [ 4,  6,  5,  5,  5,  5],
       [ 5,  7,  8,  6,  6,  6],
       [ 6,  7,  7,  7,  7,  7],
       [ 6,  9, 11,  8,  8,  8],
       [ 8, 10,  9,  9,  9,  9],
       [ 9, 10, 10, 10, 10, 10],
       [ 8, 12, 11, 11, 11, 11],
       [ 4, 11, 13, 12, 12, 12],
       [12, 14, 13, 13, 13, 13],
       [ 2, 13, 15, 14, 14, 14],
       [14, 16, 15, 15, 15, 15],
       [15, 16, 16, 16, 16, 16],
       [18, 17, 17, 17, 17, 17],
       [17, 19, 18, 18, 18, 18],
       [18, 20, 19, 19, 19, 19],
       [19, 21, 21, 22, 20, 20],
       [20, 20, 21, 21, 21, 21],
       [20, 22, 22, 22, 22, 22],
       [24, 28, 23, 23, 23, 23],
       [23, 25, 24, 24, 24, 24],
       [24, 26, 25

In [15]:
loaded_data[1]['element'].shape

(17,)

In [31]:
# [CH3:18][CH2:19][O:20][C:21](=[O:22])[Cl:23].[CH3:1][O:2][c:3]1[cH:4][c:5]2[n:6][c:7]([NH2:8])[c:9]([O:10][CH3:11])[n:12][c:13]2[cH:14][c:15]1[O:16][CH3:17].[cH:24]1[cH:25][cH:26][n:27][cH:28][cH:29]
loaded_data[0]['src_charge']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [18]:
import re
line = "[CH3:18][CH2:19][O:20][C:21](=[O:22])[Cl:23].[CH3:1][O:2][c:3]1[cH:4][c:5]2[n:6][c:7]([NH2:8])[c:9]([O:10][CH3:11])[n:12][c:13]2[cH:14][c:15]1[O:16][CH3:17].[cH:24]1[cH:25][cH:26][n:27][cH:28][cH:29]1>>CCOC(=O)Nc1nc2cc(OC)c(OC)cc2nc1OC 8;;21-23-1.0;21-8-1.0"
rxn = line.split()[0].split(">>")
src = rxn[0]
tgt = rxn[1]

print(src)
print(tgt)

[CH3:18][CH2:19][O:20][C:21](=[O:22])[Cl:23].[CH3:1][O:2][c:3]1[cH:4][c:5]2[n:6][c:7]([NH2:8])[c:9]([O:10][CH3:11])[n:12][c:13]2[cH:14][c:15]1[O:16][CH3:17].[cH:24]1[cH:25][cH:26][n:27][cH:28][cH:29]1
CCOC(=O)Nc1nc2cc(OC)c(OC)cc2nc1OC


In [19]:
pattern = re.compile(":(\d+)\]")
pattern.findall(tgt)

[]

In [21]:
from rdkit import Chem
src_mols = src.split('.')
src_mols = [Chem.MolFromSmiles(item) for item in src_mols]
src_mols

In [22]:
tgt_mols = [Chem.MolFromSmiles(item) for item in tgt.split(".")]
tgt_mols

In [24]:
ranges = []
for mol in src_mols:
    lower = 999
    upper = 0
    for atom in mol.GetAtoms():
        print(atom.GetAtomMapNum())
        lower = min(lower, atom.GetAtomMapNum()-1)
        upper = max(upper, atom.GetAtomMapNum())
        print(lower, upper)
        print('-'*10)
    ranges.append((lower, upper))

18
17 18
----------
19
17 19
----------
20
17 20
----------
21
17 21
----------
22
17 22
----------
23
17 23
----------
1
0 1
----------
2
0 2
----------
3
0 3
----------
4
0 4
----------
5
0 5
----------
6
0 6
----------
7
0 7
----------
8
0 8
----------
9
0 9
----------
10
0 10
----------
11
0 11
----------
12
0 12
----------
13
0 13
----------
14
0 14
----------
15
0 15
----------
16
0 16
----------
17
0 17
----------
24
23 24
----------
25
23 25
----------
26
23 26
----------
27
23 27
----------
28
23 28
----------
29
23 29
----------


In [26]:
ranges

[(17, 23), (0, 17), (23, 29)]

In [40]:
atom

In [43]:
# Atom: [CH3:18][CH2:19][O:20][C:21](=[O:22])[Cl:23]
atom = src_mols[0].GetAtoms()[4]
for j,b in enumerate(atom.GetBonds()):
    print(j, b.GetBeginAtomIdx(), b.GetEndAtomIdx())
    print(b.GetBeginAtomIdx() + b.GetEndAtomIdx() - atom.GetIdx())
    print(b.GetBondType())

0 3 4
3
DOUBLE
